References:
- https://pythonhosted.org/PyDrive/quickstart.html

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


# gauth = GoogleAuth()
# gauth.CommandLineAuth()

# drive = GoogleDrive(gauth)
gauth = GoogleAuth()
# Try to load saved client credentials
gauth.LoadCredentialsFile("credentials.json")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("credentials.json")

drive = GoogleDrive(gauth)


2024-02-12 00:39:47,757 INFO: file_cache is only supported with oauth2client<4.0.0


In [5]:
# Read file from GDrive in the folder "slides" (with id folder_id)
import os
from dotenv import load_dotenv
load_dotenv()


folder_id = os.environ["FOLDER_ID"]

file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList() 
new_files = []
for file in file_list:
    if not os.path.isfile(f"{file['title']}"): 
        # print("title: %s, id: %s" % (file["title"],file["id"]))
        file.GetContentFile(file["title"])
        new_files.append(file)

# new_files


In [3]:
import PyPDF2


transcriptions = [] 
embeddings = {}

for file in new_files:
    pdfReader = PyPDF2.PdfReader(file["title"])

    count = len(pdfReader.pages)
    output = ""
    embeddings[file["title"]] = {"text":[]}
    for i in range(count):
        pageObj = pdfReader.pages[i]
       
        extr = pageObj.extract_text()
        embeddings[file["title"]]["text"].append(extr)
        output += "\n" + extr
        
    transcriptions.append(output)
print(embeddings)
    

{'DeepLearning-chapter-10.pdf': {'text': ['C h a p t e r 1 0\nS e qu e n ce Mo d e l i n g: Recurren t\nan d Recursiv e N e t s\nRecurrentneuralnetworksorRNNs( ,)areafamilyof Rumelhart e t a l .1986a\nneuralnetworksforprocessingsequentialdata.Muchasaconvolutionalnetwork\nisaneuralnetworkthatisspecializedforprocessingagridofvalues Xsuchas\nanimage,arecurrentneuralnetworkisaneuralnetworkthatisspecializedfor\nprocessingasequenceofvaluesx( 1 ), . . . ,x( ) τ.Justasconvolutionalnetworks\ncanreadilyscaletoimageswithlargewidthandheight,andsomeconvolutional\nnetworkscanprocessimagesofvariablesize,recurrentnetworkscanscaletomuch\nlongersequencesthanwouldbepracticalfornetworkswithoutsequence-based\nspecialization.Mostrecurrentnetworkscanalsoprocesssequencesofvariable\nlength.\nTogofrommulti-layernetworkstorecurrentnetworks,weneedtotakeadvan-\ntageofoneoftheearlyideasfoundinmachinelearningandstatisticalmodelsof\nthe1980s:sharingparametersacrossdiﬀerentpartsofamodel.Parametersharing\nmakesitpossib

In [10]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
import io
import os

transcriptions = [];
emb = {"source":[],"page":[],"content":[]}
for file in new_files:
    embeddings[file["title"]] = {"text":[]}
    fp = open(file["title"], 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    print(type(retstr))
    codec = 'utf-16'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    full_file = ""
    page_no = 0
    for pageNumber, page in enumerate(PDFPage.get_pages(fp)):
        if pageNumber == page_no:
            interpreter.process_page(page)

            data = retstr.getvalue()
            data = ' '.join(data.splitlines())
            full_file += data + "\n"
            data = data.split(". ")
            for e in data:
                emb["source"].append(file["title"])
                emb["page"].append(pageNumber)
                emb["content"].append(data)
                #embeddings[file["title"]]["text"].append(data)
            retstr.truncate(0)
            retstr.seek(0)
        page_no += 1
    transcriptions.append(full_file)

#text = extract_text("DeepLearning-chapter-3.pdf")


<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>
<class '_io.StringIO'>


In [13]:
print(len(emb["source"]))


9571


In [42]:
import openai
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
from math import ceil


api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=api_key)
responses = []

for t in transcriptions:
    context = t
    i = 0
    chunks = []

    for i in range(ceil(len(t)/4097)):
        chunks.append(t[i*4097:i*4097+4097])

    for c in chunks:
        context = c
        question = "The text above is the result of the transcription of slides in the PDF file format. Remove chapter numbers, slides numbers and fix the text where there are line-breaks or incomplete formulas . Once you do that generate meaningful questions on the text and the respective answers. Plese reply in the JSON format {'questions':<questions generated>,'answers':<answers generated>}. DO NOT write anything else than the requested JSON and remember to write the full elaborated content and not just one part."
        #question = "The text above is the result of the transcription of slides in the PDF file format. Remove chapter names and slides numbers and rephrase the sentences. Once you do that generate 3 meaningful questions based on the new text and the respective answers. As for the reply, follow the following template FOR EACH pair of question and the respective answer: '[INST] <question> [/INST] <answer>'  and so on, let's call this template a 'block'.  NEVER use newlines other than separating blocks and NEVER write anything that is not formatted as the proposed template. DO NOT write anything else than the requested blocks and make sure everything is formatted correctly."
        # response = openai.Completion.create(
        # engine="gpt-3.5-turbo",
        prompt=f"\nContext: {context}\nQuestion: {question}"
        # )
        # answer = response.choices[0].text.strip()
        # print(answer)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )
        
        print(response.choices[0].message.content)
        responses.append(response.choices[0].message.content)


{
  "questions": [
    "What are recurrent neural networks?",
    "Why are recurrent networks specialized for processing sequences?",
    "What is the advantage of parameter sharing in recurrent neural networks?",
    "Give an example of how recurrent neural networks can recognize information at different positions within a sequence.",
    "How do recurrent neural networks differ from fully connected feedforward networks?",
    "What is the concept of parameter sharing in recurrent neural networks?",
    "What is the difference between the parameter sharing in recurrent networks and convolutional networks?",
    "How do recurrent neural networks handle sequence length and time in practice?"
  ],
  "answers": [
    "Recurrent neural networks (RNNs) are a family of neural networks for processing sequential data.",
    "Recurrent networks are specialized for processing sequences because they can scale to much longer sequences than networks without sequence-based specialization and can als

231

In [49]:
# Generate instruction set
import json
import pandas as pd

row_result = {"prompt":[],"questions":[],"answers":[]}

for i, r in enumerate(responses):
    try:
        tmp = json.loads(r)
       

        for j in range(len(tmp["questions"])):
            if type(tmp["questions"][j])!=dict:
                instr = f"<s> [INST] {tmp['questions'][j]} [/INST] {tmp['answers'][j]} </s>"
                row_result["questions"].append(tmp['questions'][j])
                row_result["answers"].append(tmp['answers'][j])
                row_result["prompt"].append(instr)
            else:
                if "question" in tmp['questions'][j] and "answer" in tmp['question'][j]:
                    instr = f"<s> [INST] {tmp['questions'][j]['question']} [/INST] {tmp['questions'][j]['answer']} </s>"
                    row_result["questions"].append(tmp['questions'][j])
                    row_result["answers"].append(tmp['answers'][j])
                    row_result["prompt"].append(instr)
        
                # print(instr)
    except:
        pass

instructions = pd.DataFrame(row_result)
pd.set_option('display.max_colwidth', None)

for index, row in instructions.iterrows():
    print(row["prompt"])


# print(instructions.iloc[0]["Instructions"])


<s> [INST] What are recurrent neural networks? [/INST] Recurrent neural networks (RNNs) are a family of neural networks for processing sequential data. </s>
<s> [INST] Why are recurrent networks specialized for processing sequences? [/INST] Recurrent networks are specialized for processing sequences because they can scale to much longer sequences than networks without sequence-based specialization and can also handle sequences of variable length. </s>
<s> [INST] What is the advantage of parameter sharing in recurrent neural networks? [/INST] The advantage of parameter sharing in recurrent neural networks is that it allows the model to generalize across examples of different forms (such as sequence lengths) and share statistical strength across different sequence lengths and positions in time. </s>
<s> [INST] Give an example of how recurrent neural networks can recognize information at different positions within a sequence. [/INST] For example, if the task is to extract the year in whic

In [50]:
len(instructions)

915

In [7]:
import hopsworks

project = hopsworks.login()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/314521


In [14]:
import pandas as pd

# emb = {"source":[],"page":[],"content":[]}
# for e in embeddings:
#     for idx,t in enumerate(embeddings[e]["text"]):
#         emb["source"].append(e)
#         emb["page"].append(idx+1)
#         emb["content"].append(t)


embedding_df = pd.DataFrame(emb)

print(embedding_df)
    
    
    

                           source  page  \
0     DeepLearning-chapter-10.pdf     0   
1     DeepLearning-chapter-10.pdf     0   
2     DeepLearning-chapter-10.pdf     0   
3     DeepLearning-chapter-10.pdf     0   
4     DeepLearning-chapter-10.pdf     0   
...                           ...   ...   
9566          01_introduction.pdf    78   
9567          01_introduction.pdf    79   
9568          01_introduction.pdf    79   
9569          01_introduction.pdf    80   
9570          01_introduction.pdf    80   

                                                content  
0     [Chapter 10  Sequence Modeling: Recurrent and ...  
1     [Chapter 10  Sequence Modeling: Recurrent and ...  
2     [Chapter 10  Sequence Modeling: Recurrent and ...  
3     [Chapter 10  Sequence Modeling: Recurrent and ...  
4     [Chapter 10  Sequence Modeling: Recurrent and ...  
...                                                 ...  
9566  [Cross-Entropy - Example  ▶ Six tosses of a co...  
9567  [References  

In [15]:
fs = project.get_feature_store()
emb_fg = fs.get_or_create_feature_group(
    name="embeddings",
    version=3,
    primary_key=list(embedding_df), 
    description="Content of each page of each file")
emb_fg.insert(embedding_df)

Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/314521/fs/314440/fg/515417


Uploading Dataframe: 100.00% |██████████| Rows 9571/9571 | Elapsed Time: 00:39 | Remaining Time: 00:00


Launching job: embeddings_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/314521/jobs/named/embeddings_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fa287daa0d0>, None)

In [55]:
fs = project.get_feature_store()
instructions_fg = fs.get_or_create_feature_group(
    name="instructionset",
    version=4,
    primary_key=list(instructions), 
    description="Instruction Set for fine tuning of llms")
instructions_fg.insert(instructions)

Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 915/915 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: instructionset_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/314521/jobs/named/instructionset_4_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fa6f7aa9e50>, None)

In [28]:
#print(embedding_df['source'].isin(['02_serverless_ml.pdf','Python']))

fs = project.get_feature_store()

True